In [ ]:
# Created by: Adam Fabo
# Date: 22.5.2022
# Created at HMU Crete
# Class: Neural Networks
# File contains script to create neural network with 1 hidden layer (Chapter 4 in documentation) 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import neurolab as nl
import pandas as pd
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os



In [ ]:
# co mozem sledovat

# zvysovat pocet hidden layers 0-3
# zvysovat pocet neuronov v hidden layers
# pocet epochs - 100 - 3000 cca po stovkach
# error pri trenovani
# error pri sample vstupoch 
# cas - kolko trvalo natrenovat neuronku

# a monzo este nejako to spriemerovat?

In [ ]:
# load data
data = pd.read_csv('data_banknote_auth_trimmed.txt', sep=",", header=None)

# data description
# 1. variance of Wavelet Transformed image (continuous)
# 2. skewness of Wavelet Transformed image (continuous)
# 3. curtosis of Wavelet Transformed image (continuous)
# 4. entropy of image (continuous)
# 5. class (integer)

data.columns = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]

data.head()

In [ ]:
# get class as separate array
target = data.copy()["Class"]
target = target.to_numpy()

# two categories so 2 neurons, change 0 to [0,1] an 1 to [1,0]
banknotes = {0: [0,1], 1: [1,0]}
target = [ banknotes[number] for number in target]

target = np.array(target)


data = data.drop(columns = ["Class"])
data = data.to_numpy()


In [ ]:
# one hidden layer

# test size = 30%
# transfer func = LogSig
# training func = Resilient Backpropagation
# learning goal = 1e-5
# learning rate = 0.05

min_neurons = 1
max_neurons = 10

min_epochs = 100
max_epochs = 3000
epoch_step = 100

# for statistical errors
repeats = 10


In [ ]:


for num_of_neurons in range(min_neurons,max_neurons+1):
     # split the data and create the NN before training 
    for round_num in range(10):
        
        # split the dataset
        X_train, X_test, y_train, y_test = train_test_split(data,target,test_size=0.3)

        # scale values to range (0,1)
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train = min_max_scaler.fit_transform(X_train)
        X_test  = min_max_scaler.fit_transform(X_test)



        # create NN with num_of_neurons neurons in hidden layer and 2 in output
        net = nl.net.newff(nl.tool.minmax(X_train),[num_of_neurons,2])
        net.layers[-1].transf = nl.trans.LogSig()
        net.layers[ 0].transf = nl.trans.LogSig()


        # set train function
        net.trainf = nl.train.train_rprop

        error = []
        train_time = 0

        # Training of NN partially
        for num_of_epochs in range(min_epochs,max_epochs + epoch_step, epoch_step):
            print("Neurons: " + str(num_of_neurons) + ", Epochs: " + str(num_of_epochs) + ", Round: " +  str(round_num))


            #create dataframe to which data will be stored
            final_data = pd.DataFrame(columns = ["Round","Neurons","Epochs","Accuracy on training","Accuracy on test","Training time","Error"])

            #for round_num in range(repeats):
            #    print("Neurons: " + str(num_of_neurons) + ", Epochs: " + str(num_of_epochs) + ", Round: " +  str(round_num))


            #train for 100 epochs
            start_time = time.time()
            error += net.train(X_train,y_train,epochs = epoch_step, show = 100, lr = 0.05, goal=1e-5)
            train_time += time.time() - start_time
            


            # test training data
            out = net.sim(X_train)
            out = np.around(out)
            correct = (out == y_train).all(axis = 1)
            accuracy_training = (np.sum(correct)/len(out))*100


            # test test data
            out = net.sim(X_test)
            out = np.around(out)
            correct = (out == y_test).all(axis = 1)
            accuracy_test = (np.sum(correct)/len(out))*100


            df = pd.DataFrame([[round_num,num_of_neurons,num_of_epochs,accuracy_training,accuracy_test,train_time,error]],columns = ["Round","Neurons","Epochs","Accuracy on training","Accuracy on test","Training time","Error"])
            final_data = pd.concat([final_data,df])


            filename = "partial_train/" + str(num_of_neurons) + "_neurons/"+ str(num_of_epochs) + "_epochs.csv"

            # if file exists, only append
            if os.path.isfile(filename):
                final_data.to_csv(filename,mode="a",header=False)
            else:
                final_data.to_csv(filename)
  
        
        
        
        
        
